<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Autoencoders

Um **Autoencoder** é um tipo de rede neural artificial usada para aprendizado não supervisionado de representações eficientes (codificações). O objetivo de um autoencoder é aprender uma representação (encoding) para um conjunto de dados, normalmente para redução de dimensionalidade ou extração de características.

Ele consiste em duas partes principais:
1.  **Encoder**: Uma rede que comprime os dados de entrada em uma representação latente de menor dimensão (o "encoding" ou "bottleneck").
2.  **Decoder**: Uma rede que tenta reconstruir os dados de entrada originais a partir da representação comprimida.

A rede é treinada para minimizar o **erro de reconstrução**, que é a diferença entre a entrada original e a saída reconstruída. Ao forçar os dados a passarem por um "gargalo" (bottleneck) de menor dimensão, a rede aprende as características mais importantes dos dados.

Neste notebook, vamos construir um autoencoder simples para comprimir e reconstruir imagens do famoso dataset **MNIST**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

print(f"TensorFlow versão: {tf.__version__}")
print("Bibliotecas importadas com sucesso!")

In [ ]:
# Carregando o dataset. Não precisamos dos rótulos (y_train, y_test) para treinar o autoencoder,
# mas vamos usá-los no final para visualização.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Pré-processamento:
# 1. Normalizar os pixels para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# 2. Achatar as imagens 28x28 em vetores de 784 dimensões
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(f"Formato dos dados de treino: {x_train.shape}")
print(f"Formato dos dados de teste:  {x_test.shape}")

In [ ]:
"""
Vamos construir nosso modelo usando a API Funcional do Keras, que é ótima para visualizar arquiteturas não-sequenciais como esta.

A imagem original (784 dimensões) será comprimida para uma representação latente de apenas 32 dimensões.
"""
# Tamanho da nossa representação codificada (o bottleneck)
encoding_dim = 32

# Camada de entrada
input_img = Input(shape=(784,))

# --- ENCODER ---
# "encoded" é a representação comprimida da entrada
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded) # A representação do bottleneck

# --- DECODER ---
# "decoded" é a reconstrução da imagem a partir do bottleneck
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded) # 'sigmoid' para manter os valores entre 0 e 1

# --- AUTOENCODER MODEL ---
# Este modelo mapeia uma entrada para sua reconstrução
autoencoder = Model(input_img, decoded)

# Compilando o modelo. O objetivo é minimizar a diferença entre input e output.
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()



In [ ]:
"""
Para treinar, usamos `x_train` tanto como dados de entrada quanto como o alvo da predição.
A rede aprende a gerar `x_train` a partir de `x_train`.
"""
history = autoencoder.fit(x_train, x_train, # Note: input e target são os mesmos
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                verbose=1)

# Plotando a perda de treino e validação
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Perda de Treino')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Histórico de Perda do Modelo')
plt.ylabel('Loss')
plt.xlabel('Época')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
"""
Agora, o momento da verdade! Vamos pegar algumas imagens do conjunto de teste,
passá-las pelo autoencoder e comparar as imagens originais com as reconstruídas.
"""

# Usando o modelo treinado para prever (reconstruir) as imagens de teste
decoded_imgs = autoencoder.predict(x_test)

# Plotando os resultados
n = 10  # Quantidade de dígitos para exibir
plt.figure(figsize=(20, 4))
for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title("Original", loc='left')

    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title("Reconstruída", loc='left')

plt.suptitle("Comparação: Imagens Originais vs. Reconstruídas", fontsize=16)
plt.show()

"""
**Observação:** As imagens reconstruídas são um pouco borradas, o que é esperado.
Isso acontece porque o autoencoder é um modelo com perdas (*lossy*).
 No entanto, ele conseguiu capturar as características essenciais para
 reconstruir uma versão reconhecível do dígito original, mesmo após a
 compressão massiva de 784 para 32 dimensões.
"""

In [ ]:
"""
O que a representação comprimida (espaço latente) aprendeu?
Se construirmos um modelo onde o `encoding_dim` é 2, podemos plotar a representação de cada dígito em um gráfico 2D e ver se o autoencoder aprendeu a agrupar dígitos semelhantes.
"""
# Criando um modelo separado apenas para o encoder
encoder = Model(input_img, encoded)

# Passando as imagens de teste pelo encoder para obter suas representações no espaço latente
x_test_encoded = encoder.predict(x_test)

# Plotando o espaço latente 2D
plt.figure(figsize=(12, 10))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test, cmap='viridis')
plt.colorbar(label='Rótulo do Dígito')
plt.title('Visualização do Espaço Latente do Autoencoder (32D -> 2D)')
plt.xlabel('Dimensão Latente 1')
plt.ylabel('Dimensão Latente 2')
plt.grid(True)
plt.show()

"""
**Conclusão:** Mesmo sem usar os rótulos durante o treino, o autoencoder aprendeu a organizar o espaço latente de forma que dígitos visualmente semelhantes fiquem próximos uns dos outros. Isso demonstra sua poderosa capacidade de extração de características de forma não supervisionada. Vemos clusters distintos para os diferentes dígitos, o que é um resultado impressionante.
"""
